In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
import tensorflow as tf
import numpy as np

import time

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstarping_partitions_full_int_q,
    )

In [3]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-07-26 12:29:48.507827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 12:29:48.508505: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-07-26 12:29:48.606327: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-26 12:29:48.610892: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [4]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

In [5]:
BASE_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 47996, 4)          24        
                                                                 
 max_pooling1d_21 (MaxPooli  (None, 5999, 4)           0         
 ng1D)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 5995, 4)           84        
                                                                 
 max_pooling1d_22 (MaxPooli  (None, 749, 4)            0         
 ng1D)                                                           
                                                                 
 conv1d_23 (Conv1D)          (None, 745, 8)            168       
                                                                 
 max_pooling1d_23 (MaxPooli  (None, 93, 8)           

### Convert to TF Lite format

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpm537xz55/assets


INFO:tensorflow:Assets written to: /tmp/tmpm537xz55/assets
2024-07-26 12:31:28.824838: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-26 12:31:28.824908: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-26 12:31:28.825550: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpm537xz55
2024-07-26 12:31:28.827251: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-26 12:31:28.827297: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpm537xz55
2024-07-26 12:31:28.831456: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-26 12:31:28.832776: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-26 12:31:28.887968: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [11]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/quantized/cnn_time_series.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.61%
Precision: 98.28%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9748849299738644

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9741021979962435

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9763907459574618

Time for Test dataset:
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.006 seconds


In [12]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 44.035 Kilobytes


### Dynamic range quantization

In [13]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp5vn8d0sb/assets


INFO:tensorflow:Assets written to: /tmp/tmp5vn8d0sb/assets
2024-07-26 12:34:58.800308: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-26 12:34:58.800399: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-26 12:34:58.800625: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp5vn8d0sb
2024-07-26 12:34:58.802152: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-26 12:34:58.802221: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp5vn8d0sb
2024-07-26 12:34:58.808060: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-26 12:34:58.870407: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp5vn8d0sb
2024-07-26 12:34:58.894627: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.61%
Precision: 98.28%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9748849299738644

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9710629426371107

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9782485108394106

Time for Test dataset:
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.006 seconds


In [15]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 17.953 Kilobytes


### Float16 quantization

In [16]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp1f7yw6vw/assets


INFO:tensorflow:Assets written to: /tmp/tmp1f7yw6vw/assets
2024-07-26 12:35:28.787421: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-26 12:35:28.787504: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-26 12:35:28.787739: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp1f7yw6vw
2024-07-26 12:35:28.788890: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-26 12:35:28.788904: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp1f7yw6vw
2024-07-26 12:35:28.792068: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-26 12:35:28.842869: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp1f7yw6vw
2024-07-26 12:35:28.859932: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [17]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.61%
Precision: 98.28%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9748849299738644

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9750120747420757

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9772796001626193

Time for Test dataset:
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.006 seconds


In [18]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 27.406 Kilobytes


### Float fallback

In [31]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpf_iq0fan/assets


INFO:tensorflow:Assets written to: /tmp/tmpf_iq0fan/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-26 12:44:39.687376: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-26 12:44:39.687476: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-26 12:44:39.688054: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpf_iq0fan
2024-07-26 12:44:39.689511: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-26 12:44:39.689534: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpf_iq0fan
2024-07-26 12:44:39.693674: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-26 12:44

In [32]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_float_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.12%
Recall: 96.61%
Precision: 97.85%
F1-score: 97.23%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9727362070633815

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9696877052104296

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.42%
Recall: 96.29%
Precision: 98.88%
F1-score: 97.57%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9746910654603242

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9761427819897763

Time for Test dataset:
Accuracy: 98.42%
Recall: 96.29%
Precision: 98.88%
F1-score: 97.57%

Time to make a prediction for a single data point
Mean: 0.004 seconds
Max: 0.016 seconds


### Full integer quantization

In [24]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

full_int_model = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp36_7qvb7/assets


INFO:tensorflow:Assets written to: /tmp/tmp36_7qvb7/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-26 12:39:50.128644: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-26 12:39:50.128704: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-26 12:39:50.128933: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp36_7qvb7
2024-07-26 12:39:50.131091: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-26 12:39:50.131114: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp36_7qvb7
2024-07-26 12:39:50.136474: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-26 12:39

In [25]:
interpreter = tf.lite.Interpreter(model_content=full_int_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [27]:
import pathlib

tflite_models_dir = pathlib.Path("../time_series_models_from_notebooks/cnn/quantized/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_full_int_q_file = tflite_models_dir/"cnn_time_series_full_int_q.tflite"
tflite_model_full_int_q_file.write_bytes(full_int_model)

19360

In [30]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_full_int_q_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_full_int_q_file), "KB")

Validation dataset:
Accuracy: 89.35%
Recall: 69.28%
Precision: 99.39%
F1-score: 81.65%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.8167337655709972

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.820223260281808

Test dataset:
Accuracy: 87.51%
Recall: 62.23%
Precision: 99.65%
F1-score: 76.61%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.7655054803839321

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.7664036440754879

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.004 seconds
Max: 0.009 seconds
Min: 0.003 seconds
File size: 18.906 Kilobytes
